In [ ]:
import pandas as pd
import matplotlib as m
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import numpy as np
from pathlib import Path
import json
import math
from textwrap import wrap
from matplotlib import ticker
import os

In [ ]:
export_format = "pdf"

In [ ]:
workload_display_name = {
    "addpd": "128b SSE Add (1 Thread)",
    "busywait": "Busy Waiting",
    "compute": "Dot Product (1 Thread)",
    "idle": "Sleep",
    "matmul": "DGEMM (1 Thread)",
    "memorycopy": "Memory Copy (32 Threads)",
    "memoryread": "Memory Read (32 Threads)",
    "memorywrite": "Memory Write (32 Threads)",
    "memorycopy_4": "Memory Copy (4 Threads)",
    "memoryread_4": "Memory Read (4 Threads)",
    "memorywrite_4": "Memory Write (4 Threads)",
    "mulpd": "128b SSE Mult. (1 Thread)",
    "stream_add": "STREAM Add (32 Threads)",
    "stream_add_4": "STREAM Add (4 Threads)",
    "stream_combined": "STREAM (32 Threads)",
    "stream_combined_4": "STREAM (4 Threads)",
    "stream_copy_1": "STREAM Copy (1 Thread)",
    "stream_copy_2": "STREAM Copy (2 Threads)",
    "stream_copy_4": "STREAM Copy (4 Threads)",
    "stream_copy_16": "STREAM Copy (16 Threads)",
    "stream_copy_24": "STREAM Copy (24 Threads)",
    "stream_copy_32": "STREAM Copy (32 Threads)",
    "stream_copy_8": "STREAM Copy (8 Threads)",
    "stream_scale": "STREAM Scale (32 Threads)",
    "stream_scale_4": "STREAM Scale (4 Threads)",
    "stream_triad": "STREAM Triad (32 Threads)",
    "stream_triad_4": "STREAM Triad (4 Threads)"

}

In [ ]:
energy_errors = {} 

In [ ]:
"""
Exports the specified figure to a file

:param fig Figure to save
:name Path prefix (without extension) to save the figure to.
"""
def savefig(fig, name):
    fig.savefig(name + "." + export_format, format=export_format, bbox_inches="tight")


In [ ]:
"""
Stores the specified legend object to a file

:param legend legend to save
:param filename name of the file to save to
"""
def export_legend(legend, filename="legend.pdf", expand=[-5,-5,5,5]):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename, dpi="figure", format=export_format, bbox_inches=bbox)

In [ ]:
"""
Reads the raw reference measurements from the specified file and computes the current.

:param path Path to the file to load data from
:param config Configuration object for this memory configuration
"""
def read_reference(path, config):
    df = pd.read_csv(path, names=[f"v{i}" for i in [0, 1, 2, 3, 4, 5, 6, 7]], header=None)
    channel_config = config["channels"]
    dimm_channels = {}
    
    for channel in channel_config:
        key = str(channel["riser"]) + "-" + channel['name']
        multiplier = 1.0 if "riser_multipliers" not in config else config["riser_multipliers"][channel["riser"]]
        df_voltage = df[f"v{channel['voltage_channel']}"] / channel["voltage_gain"]
        df_current = df[f"v{channel['voltage_drop_channel']}"] / channel["gain"] / channel["csr_resistance"]
        df[key] = df_voltage * df_current * multiplier
        channels_per_dimm = []
        if channel["riser"] in dimm_channels:
            channels_per_dimm = dimm_channels[channel["riser"]]
        channels_per_dimm.append(key)
        dimm_channels[channel["riser"]] = channels_per_dimm
        
    return df, dimm_channels

In [ ]:
"""
Reads RAPL energy measurements from the specified file

:param path Path to the file to load data from
:param config Configuration object for this memory configuration
"""
def read_rapl(path, config):
    rapl_energy_sum = 0.0
    sockets = [0] if "sockets" not in config else config["sockets"]
    rapl_entries = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#") or line == "":
                continue
            split = line.split(",")
            if len(split) < 5:
                continue
            timestamp = float(split[0])
            delay = float(split[1])
            event_name = split[3]
            socket = split[2]
            energy = split[4]
            power = split[5]
            if "ram" not in event_name:
                continue

            if int(socket.replace("N", "")) not in sockets:
                continue

            column = "RAPL_" + socket + "_" + event_name
            if column in rapl_entries:
                d = rapl_entries[column]
            else:
                d = []
            rapl_energy_sum += float(energy)
            d.append({"time": timestamp * 1000, "power": float(power), "energy": float(energy), "length": delay})
            rapl_entries[column] = d
    return rapl_entries, rapl_energy_sum

In [ ]:
"""
Generates a single phase plot (RAPL and reference measurements over time) for the specified memory configuration, workload, and test system.

:param mid ID of the experiment
:param source_dir Path to the workload-specific folder in the memory configuration folder (e.g. icelake/ddr4-32gb-3200/addpd/)
:param name internal name for the workload
:param config configuration object for the memory configuration
:param base_folder Folder in which the generated figure will be generated
"""
def create_phase_plot(mid: str, source_dir, name: str, config, base_folder):
    avg = 25 # number of samples in running average
    fig, ax = plt.subplots(figsize=(9, 3))
    
    raw_name = name
    if name in workload_display_name:
        name = workload_display_name[name]

    print("Opening path", source_dir, mid)
    ref_df, dimm_channels = read_reference(Path(source_dir, mid + ".csv"), config)
    rapl_entries, rapl_energy_sum = read_rapl(Path(source_dir, mid + '_perf.txt'), config)
    
    max_time = 0.0
    rapl_colors = ["tab:orange", "tab:blue"]
    dimm_colors = ["tab:orange", "tab:blue"]
    additional_colors = ["magenta", "cyan", "brown"]

    rapl_energy_total = 0.0
    # visualize rapl entries
    for i, (event_name, items) in enumerate(rapl_entries.items()):
        df_rapl = pd.DataFrame.from_records(items)
        ax.plot(df_rapl["time"], df_rapl["power"].rolling(5, min_periods=1).mean(), color=rapl_colors[i], linewidth=1, label=f"RAPL: Socket {0 if 'N0' in event_name else 1}")
        max_time = np.max([np.max(df_rapl["time"]), max_time])
        rapl_energy_total += np.sum(df_rapl["energy"])

    # visualize reference entries
    all_channels = []
    total_energy = 0.0
    
    dur_ms = int(np.floor(max_time))
    for dimm_index in dimm_channels:
        ax.plot(ref_df.index, (ref_df[dimm_channels[dimm_index]].sum(axis=1)).rolling(avg, min_periods=1).mean(),
                color=dimm_colors[dimm_index],
                linewidth=1,
                linestyle="dashed",
                label="Reference: " + config['riser_names'][dimm_index])
        x_array = ref_df.index[:dur_ms]
        y_array = ref_df[dimm_channels[dimm_index]].sum(axis=1)[x_array]
        energy = np.trapz(y_array, dx=0.001)  # 1 ms granularity
        total_energy += energy
        avg_power = energy / (dur_ms / 1000)
        for d in dimm_channels[dimm_index]:
            all_channels.append(d)
    
    rapl_relative_error = rapl_energy_total / total_energy
    
    if name not in energy_errors:
        energy_errors[name] = {}
    
    system_tag = "icelake" if "icelake" in source_dir else "broadwell"
    if system_tag not in energy_errors[name]:
        energy_errors[name][system_tag] = {}
        
    energy_errors[name][system_tag][source_dir] = rapl_relative_error
        
    # print(mid, source_dir, total_energy, rapl_energy_total, f"{dur_ms/1000:.4f}s", f"{rapl_relative_error * 100:.2f}%")
    
    # visualize sum of reference measurements
    if "plot_sum" in config and config["plot_sum"]:
        ax.plot(ref_df.index, (ref_df[all_channels].sum(axis=1)).rolling(avg, min_periods=1).mean(),
                color="tab:green",
                linestyle="dotted",
                linewidth=1,
                label="Total Reference Measurements")
        
    # visualize baselines (optional)
    if "baselines" in config:
        for ind, baseline_conf in enumerate(config["baselines"]):
            ax.axhline(y=baseline_conf["power"], color=additional_colors[ind], linestyle='-.', linewidth=1, label=baseline_conf["label"])
    
    

    #ax.axvline(x=max_time, color="black")
    ax.set_xlim(right=min(max_time + 500, ax.get_xlim()[1]))
    ax.set_ylim(bottom=max(0, ax.get_ylim()[0]))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))

        
    tr = 1000
    if ax.get_xlim()[1] >= 20000:
        tr = 5000
    elif ax.get_xlim()[1] >= 6000:
        tr = 2000
    ax.set_xticks(np.arange(0, ax.get_xlim()[1], tr))
    ax.set_xticks(np.arange(0, ax.get_xlim()[1], 1000), minor=True)
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Power [W]")
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, a: f"{int(x/1000)}"))
    ax.set_xlim(left=0, right=max_time)
    ax.grid(which="minor", alpha=.3)
    ax.grid(which="major", alpha=.8)
    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

    # Put a legend below current axis
    ax.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=2, prop={'size': 12})
    
    savefig(fig, base_folder + "/" + raw_name)
    plt.close(fig)

In [ ]:
"""
Generates a phase plot (RAPL and reference measurements over time) for each memory configuration and workload for the given test system.

:param root_folder Path to the result folder for the test system (e.g., 'icelake/' or 'broadwell/')
:param base_subfolders List of subfolders corresponding to memory configurations
"""
def visualize_phase_plots(root_folder, base_subfolders):
    for base_subfolder in base_subfolders: # iterate to all memory configurations
        base_folder = root_folder + base_subfolder
        
        if Path(base_folder, "config.json").exists():
            with open(Path(base_folder, "config.json"), "r") as f:
                config = json.load(f)
        else:
            continue

        for folder in Path(base_folder).iterdir(): # iterate through all experiments
            name = str(folder).split(os.sep)[-1]
            subfolder = Path(folder)
            if subfolder.is_file():
                continue
            if str(subfolder).split(os.sep)[-1].startswith("-"):
                continue
            for file in subfolder.iterdir():
                if "_perf.txt" in str(file):
                    print(f"Generating phase plot for system: {root_folder} Memory Configuration: {base_folder} Workload: {name}")
                    mid = str(file).split(os.sep)[-1].split("_")[0]
                    create_phase_plot(mid, str(subfolder), name=name, config=config, base_folder=base_folder)


In [ ]:
"""Generates a scatter plot correlating reference and RAPL measurements for each memory configuration on a test system.
Used for Figures 10 - 15 in the paper.

:param root_folder Path to the root results folder for the system ('icelake/' or 'broadwell')
:param base_subfolders List of subfolders corresponding to memory configurations
:param extra_legend Whether to place the legend in a separate file
:param legend_cols Number of columns in the legend
"""
def visualize_scatter_plot(root_folder, base_subfolders, extra_legend=False, legend_cols=3):
    
    colors = ["#FF0000", "#00FF00", "#0000FF", "#000000", "#FF00FF", "#00FFFF", "#FFA500", "#800080", "#008000", "#800000", "#808000", "#008080", "#808080", "#C0C0C0", "#FFD700", "#B22222", "#228B22", "#32CD32", "#87CEEB", "#40E0D0", "#D2691E", "#DC143C", "#FF1493", "#A0522D", "#800080"]
    markers = list(filter(lambda x: x != "none" and x != "None" and x != "" and x != " ",  MarkerStyle.markers))

    # iterate through all memory configurations
    for base_subfolder in base_subfolders:
        base_folder = root_folder + base_subfolder
        
        combined_df = pd.DataFrame()
        corr_fig, corr_ax = plt.subplots(figsize=(8,8))

        if Path(base_folder, "config.json").exists():
            with open(Path(base_folder, "config.json"), "r") as f:
                config = json.load(f)
        else:
            continue
            
          
        sockets = [0] if "sockets" not in config else config["sockets"]
        index = 0
        # iterate through all workloads
        for folder in sorted(Path(base_folder).iterdir(), key=str):
            name = str(folder).split(os.sep)[-1]
            subfolder = Path(folder)
            if subfolder.is_file():
                continue
            if str(subfolder).split(os.sep)[-1].startswith("-"):
                continue
            for file in subfolder.iterdir():
                if "_perf.txt" in str(file):
                    mid = str(file).split(os.sep)[-1].split("_")[0]
                    
                    if name in workload_display_name:
                        name = workload_display_name[name]
                    source_dir = str(subfolder)
                    ref_df, dimm_channels = read_reference(Path(source_dir, mid + ".csv"), config)
                    rapl_entries, rapl_energy_sum = read_rapl(Path(source_dir, mid + '_perf.txt'), config)

                    all_channels = []
                    total_energy = 0.0
                    for dimm_index in dimm_channels:
                        for d in dimm_channels[dimm_index]:
                            all_channels.append(d)

                    align = "200ms"
                    for dimm_index in dimm_channels:
                        ref_df["power_daq_" + config['riser_names'][dimm_index]] = ref_df[dimm_channels[dimm_index]].sum(axis=1)
                    ref_df["power_daq"] = ref_df[all_channels].sum(axis=1)
                    daq_df = ref_df.reset_index(names=["time"])
                    daq_df["time"] = pd.to_timedelta(daq_df["time"], unit="ms")
                    daq_df = daq_df.set_index("time").resample(align, offset=0).mean()
                    rapl_df = pd.DataFrame.from_records(rapl_entries[f"RAPL_N{sockets[0]}_ram"])[["time", "power"]].rename(
                        columns={"power": "power_rapl"})
                    rapl_df["time"] = pd.to_timedelta(rapl_df["time"], unit="ms")
                    rapl_df["time"] = rapl_df["time"].dt.floor(align)
                    rapl_df = rapl_df.set_index("time").resample(align, offset=0).mean()

                    merged = pd.merge(daq_df, rapl_df, on="time")
                    merged.sample(n=min(100, len(merged)), random_state=123).plot.scatter(x="power_daq", y="power_rapl", ax=corr_ax, label=name, s=100, c=colors[index % len(colors)],
                                        marker=MarkerStyle(markers[index % len(markers)], fillstyle="none"),  linewidths=1.5)
                    combined_df = pd.concat([combined_df, merged])
                    break
            index += 1
        corr_ax.set_xlabel("Riser Power Measurements [W]", labelpad=15)
        corr_ax.set_ylabel("RAPL Memory Power [W]", labelpad=15)
        max_lim = max(corr_ax.get_xlim()[1], corr_ax.get_ylim()[1])
        min_lim = min(corr_ax.get_xlim()[0], corr_ax.get_ylim()[0])
        corr_ax.set_xlim(left=min_lim, right=max_lim)
        corr_ax.set_ylim(bottom=min_lim, top=max_lim)
        if(min_lim < 0.5):
            min_lim = 0
        corr_ax.set_xticks(np.arange(math.ceil(corr_ax.get_xlim()[0]), math.ceil(corr_ax.get_xlim()[1]), 1))
        corr_ax.set_yticks(np.arange(math.ceil(corr_ax.get_ylim()[0]), math.ceil(corr_ax.get_ylim()[1]), 1))
        corr_ax.set_xticks(np.arange(max(0.5, np.floor(min_lim) + 0.5), min(max_lim, np.floor(max_lim) + 0.5), 1), minor=True)
        corr_ax.set_yticks(np.arange(max(0.5, np.floor(min_lim) + 0.5), min(max_lim, np.floor(max_lim) + 0.5), 1), minor=True)
        
        x_keys = np.arange(min_lim, max_lim, 0.5)
        corr_ax.plot(x_keys, x_keys, linestyle="dotted", label="Ideal Fit", linewidth=1, color="gray")  # black dashed line

        x = combined_df["power_daq"]
        y = combined_df["power_rapl"]
        slope, intercept = np.polyfit(x, y, 1)
        corr_ax.plot(x_keys, x_keys * slope + intercept, linestyle="dashed", label="Linear Fit", linewidth=1, color="red")
            
        handles, labels = corr_ax.get_legend_handles_labels()
        # sort both labels and handles by labels
        
        def extract_key(key):
            m = key[0].lower()
            if "fit" in m: # if label contains "fit", place it at the end of the legend
                return "zzzz" 
            elif "(" in m:
                tr = m.split("(")[1].split(" ")[0]
                return m.split("(")[0] + tr.zfill(3)
            else:
                return m
                
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: extract_key(t)))
        
        if not extra_legend:
            corr_ax.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=legend_cols, prop={'size': 13})
        else:
            legend = corr_ax.legend(handles, labels, loc=(1.5,1.5), ncol=5, prop={'size': 13})
            export_legend(legend, filename=base_folder + "/scatter_legend.pdf")
            corr_ax.legend().remove()

        corr_ax.grid(which="minor", alpha=.3)
        corr_ax.grid(which="major", alpha=.8)

        savefig(corr_fig, base_folder + "/scatter")
        plt.close(corr_fig)

In [ ]:
"""Generates a single scatter plot that correlates reference and RAPL measurements for different memory configurations
on a single test system. All workloads will be included in this figure but are not labeled individually. 
Used to generate Figure 7 in the paper.

:param root_folder Path to the root results folder for the system ('icelake/' or 'broadwell')
:param base_subfolders List of subfolders that will be each included as a memory configuration in the resulting figure
"""
def visualize_combined_scatter_plot(root_folder, base_subfolders):
    corr_fig, corr_ax = plt.subplots(figsize=(8,8))
    linear_fits = []

    for memory_index, base_subfolder in enumerate(base_subfolders):
        base_folder = root_folder + base_subfolder
        
        combined_df = pd.DataFrame()

        if Path(base_folder, "config.json").exists():
            with open(Path(base_folder, "config.json"), "r") as f:
                config = json.load(f)
        else:
            continue
        label = config["memory_population"]
        sockets = [0] if "sockets" not in config else config["sockets"]
        
        colors = ["tab:orange", "tab:blue", "tab:cyan", "tab:red", "tab:green"]
        
        for index, folder in enumerate(Path(base_folder).iterdir()):
            name = str(folder).split(os.sep)[-1]
            subfolder = Path(folder)
            if subfolder.is_file():
                continue
            if str(subfolder).split(os.sep)[-1].startswith("-"):
                continue
            for file in subfolder.iterdir():
                if "_perf.txt" in str(file):
                    mid = str(file).split(os.sep)[-1].split("_")[0]
                    
                    if name in workload_display_name:
                        name = workload_display_name[name]
                    source_dir = str(subfolder)
                    ref_df, dimm_channels = read_reference(Path(source_dir, mid + ".csv"), config)
                    rapl_entries, rapl_energy_sum = read_rapl(Path(source_dir, mid + '_perf.txt'), config)

                    all_channels = []
                    total_energy = 0.0
                    for dimm_index in dimm_channels:
                        for d in dimm_channels[dimm_index]:
                            all_channels.append(d)

                    align = "200ms"
                    for dimm_index in dimm_channels:
                        ref_df["power_daq_" + config['riser_names'][dimm_index]] = ref_df[dimm_channels[dimm_index]].sum(axis=1)
                    ref_df["power_daq"] = ref_df[all_channels].sum(axis=1)
                    daq_df = ref_df.reset_index(names=["time"])
                    daq_df["time"] = pd.to_timedelta(daq_df["time"], unit="ms")
                    daq_df = daq_df.set_index("time").resample(align, offset=0).mean()
                    rapl_df = pd.DataFrame.from_records(rapl_entries[f"RAPL_N{sockets[0]}_ram"])[["time", "power"]].rename(
                        columns={"power": "power_rapl"})
                    rapl_df["time"] = pd.to_timedelta(rapl_df["time"], unit="ms")
                    rapl_df["time"] = rapl_df["time"].dt.floor(align)
                    rapl_df = rapl_df.set_index("time").resample(align, offset=0).mean()

                    merged = pd.merge(daq_df, rapl_df, on="time")
                    markers = ["1", "+", ".", "p", "*"]
                    merged.sample(n=min(25, len(merged)), random_state=123).plot.scatter(x="power_daq", y="power_rapl", ax=corr_ax, label=label, s=40, c=colors[memory_index % len(colors)],
                                        marker=MarkerStyle(markers[memory_index % len(markers)], fillstyle="none"),  linewidths=1)
                    combined_df = pd.concat([combined_df, merged])
        
        x = combined_df["power_daq"]
        y = combined_df["power_rapl"]
        slope, intercept = np.polyfit(x, y, 1)
        linear_fits.append((slope, intercept))
        
    
    
    corr_ax.set_xlabel("Riser Power Measurements [W]", labelpad=15)
    corr_ax.set_ylabel("RAPL Memory Power [W]", labelpad=15)

    # x and y axis limits and tick steps
    max_lim = max(corr_ax.get_xlim()[1], corr_ax.get_ylim()[1])
    min_lim = min(corr_ax.get_xlim()[0], corr_ax.get_ylim()[0])
    if(min_lim < 0.5):
        min_lim = 0
    corr_ax.set_xlim(left=min_lim, right=max_lim)
    corr_ax.set_ylim(bottom=min_lim, top=max_lim)
    corr_ax.set_xticks(np.arange(math.ceil(corr_ax.get_xlim()[0]), math.ceil(corr_ax.get_xlim()[1]), 1))
    corr_ax.set_yticks(np.arange(math.ceil(corr_ax.get_ylim()[0]), math.ceil(corr_ax.get_ylim()[1]), 1))
    corr_ax.set_xticks(np.arange(max(0.5, np.floor(min_lim) + 0.5), min(max_lim, np.floor(max_lim) + 0.5), 1), minor=True)
    corr_ax.set_yticks(np.arange(max(0.5, np.floor(min_lim) + 0.5), min(max_lim, np.floor(max_lim) + 0.5), 1), minor=True)

    x_keys = np.arange(corr_ax.get_xlim()[0], corr_ax.get_xlim()[1], 0.5)
    corr_ax.plot(x_keys, x_keys, linestyle="dashdot", label="Ideal Fit", linewidth=1, color="black")  # black dashed line, shows ideal fit (slope = 1)
    for index, (slope, intercept) in enumerate(linear_fits):
        corr_ax.plot(x_keys, x_keys * slope + intercept, linestyle="dashed", linewidth=1, color=colors[index])
        
    corr_ax.plot([0], [0], linestyle="dashed", linewidth=1, color="gray", label="Linear Fit") # workaround for 
    
    handles0, labels0 = corr_ax.get_legend_handles_labels()
    labels = []
    handles = []
    
    # line wrap all legend entries and only show one entry for multiple legend entries with the same label
    for label, handle in zip(labels0, handles0):
        label = '\n'.join(wrap(label, 20))
        if label not in labels:
            labels.append(label)
            handles.append(handle)
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: "_" + t[0].lower() if "Fit" in t[0] else "__" + t[0].lower() if "PMM" in t[0] else t[0].lower()))
    
    # Put a legend to the right of the current axis
    corr_ax.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=3, prop={'size': 13}, scatterpoints=3)

    corr_ax.grid(which="minor", alpha=.3)
    corr_ax.grid(which="major", alpha=.8)

    savefig(corr_fig, root_folder + "/combined_scatter")
    plt.close(corr_fig)

In [ ]:
"""
Generate all phase plots for different workloads and memory configurations on the Broadwell-EP system.
These figures are not included in the paper.
"""
m.rcParams.update({"font.size": 24, "legend.fontsize": 20, 'pdf.fonttype': 42, 'ps.fonttype': 42})
visualize_phase_plots("./broadwell/", ["ddr4-2x16gb-2400-socket0"])

In [ ]:
"""
Generate all phase plots for different workloads and memory configurations on the Ice Lake-SP system.
This includes Figure 8 (icelake\ddr4-32gb-3200-socket0\idle) and Figure 9 (icelake\ddr4-32gb-3200\memorycopy) from the paper.
"""
m.rcParams.update({"font.size": 18, "legend.fontsize": 16, 'pdf.fonttype': 42, 'ps.fonttype': 42})
visualize_phase_plots("icelake/", ["ddr4-32gb-3200-socket0", "dram", "optane", "ddr4-16gb-2400", "ddr4-32gb-3200", "ddr4-2x32gb-3200"])

In [ ]:
"""
Generate all scatter plots for the different memory configurations on theBroadwel system
This includes Figures 11 - 15 from the paper
"""
m.rcParams.update({"font.size": 18, "legend.fontsize": 20, 'pdf.fonttype': 42, 'ps.fonttype': 42})
visualize_scatter_plot("broadwell/", ["ddr4-2x16gb-2400-socket0"], extra_legend=False, legend_cols=2)

In [ ]:
"""
Generate all scatter plots for the different memory configurations on the Ice Lake-SP system
This includes Figures 11 - 15 from the paper
"""
m.rcParams.update({"font.size": 24, "legend.fontsize": 20, 'pdf.fonttype': 42, 'ps.fonttype': 42})
visualize_scatter_plot("icelake/", ["ddr4-32gb-3200-socket0", "dram", "optane", "ddr4-16gb-2400", "ddr4-32gb-3200", "ddr4-2x32gb-3200"], extra_legend=True)

In [ ]:
"""
Generates the combined scatter plot for the Ice Lake-SP system (Figure 7 in the paper).
"""
m.rcParams.update({"font.size": 18, "legend.fontsize": 20, 'pdf.fonttype': 42, 'ps.fonttype': 42})
visualize_combined_scatter_plot("icelake/", ["dram", "optane", "ddr4-16gb-2400", "ddr4-32gb-3200", "ddr4-2x32gb-3200"])

In [ ]:
for workload, item in energy_errors.items():
    line = "Workload"
    for system, item2 in item.items():
        for memory, val in item2.items():
            if "ddr4-32gb-3200-socket0" in memory:
                continue
            line = line + " & " + memory.split(os.sep)[1]
    print(line)
    break
for workload, item in energy_errors.items():
    line = workload
    for system, item2 in item.items():
        for memory, val in item2.items():
            if "ddr4-32gb-3200-socket0" in memory:
                continue
            line = line + f" & {100*(val-1.0):.1f}\%"
    print(line, "\\\\")